# From Comparison to Prediction

This chapter is about training a machine learning algorithm on the similarity scores calculated in the previous chapter, in order to predict if two records are essentially the same or not.

## Set-up of the score matrix

### Data imports

In [16]:
import pandas as pd
from suricate.data.companies import getXlr
X_lr = getXlr(nrows=500)

In [17]:
X_lr[0].sample(5)

,name,street,city,postalcode,duns,countrycode
ix,,,,,,
ea464e59-6a49-4424-af64-5a3eb67e502d,adf technologies championdoor adf technologies...,6 avenue jean monnet,colomiers,31770,NaN,FR
d8fa1b69-823b-4456-b13d-5235988514ae,fako,15 peutestrae,hamburg,20539,340213235.0,DE
77c66fd9-8215-4702-945b-92fd6958b535,rg gmbh,7 im meiel,waldenbuch,71111,319932075.0,DE
76e834db-6e0c-400a-8f51-98b2652fd479,datalis,8 avenue gutenberg,portet sur garonne,31125,260332779.0,FR
0e02feec-dbf0-4693-b7bf-0b4d01629baa,heider druck gmbh,102 116 paffrather str,bergisch gladbach,51465,323117283.0,DE


### Creating the score matrix
We can pipeline and concatenate several comparators using standard scikit-learn operators

In [18]:
from suricate.lrdftransformers import FuzzyConnector, VectorizerConnector, ExactConnector
from sklearn.pipeline import FeatureUnion

In [24]:
scores = [
    ('name_vecword', VectorizerConnector(on='name', analyzer='word', ngram_range=(1,2))),
    ('name_vecchar', VectorizerConnector(on='name', analyzer='char', ngram_range=(1,3))),
    ('street_vecword', VectorizerConnector(on='street', analyzer='word', ngram_range=(1,2))),
    ('street_vecchar', VectorizerConnector(on='street', analyzer='char', ngram_range=(1,3))),
    ('city_vecchar', VectorizerConnector(on='city', analyzer='char', ngram_range=(1,3))),
    ('postalcode_exact', ExactConnector(on='postalcode')),
    ('duns_exact', ExactConnector(on='duns')),
    ('countrycode_exact', ExactConnector(on='countrycode'))
]
transformer = FeatureUnion(scores)
X_score = transformer.fit_transform(X_lr)
X_score.shape

(250000, 8)

### Pipelining for dimension reduction

In [40]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer as Imputer
from sklearn.preprocessing import Normalizer as Scaler
from sklearn.decomposition import PCA
steps = [
    ('scorer', transformer),
    ('imputer', Imputer(strategy='constant', fill_value=0)),
    ('scaler', Scaler()),
    ('pca', PCA(n_components=3))
]
preprocessing_pipeline = Pipeline(steps)
X_score_ready = preprocessing_pipeline.fit_transform(X=X_lr)
print(X_score_ready.shape)
# from seaborn import pairplot
# %matplotlib inline
# pairplot(pd.DataFrame(X_score_ready))

(250000, 3)


## Training the model

### Asking representative questions
#### By taking a sample of each cluster of pairs, we have a representative subset of the whole data set
Since we have many possibles pairs (n_rows_left * n_rows_right), we must take a representative subset of all possible pairs in order to train the model. Based on the score matrix defined at the end of the preprocessing pipeline, we can do that using clustering. We cluster the pairs according to their similarity scores.


In [61]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import QuantileTransformer
from suricate.lrdftransformers import LrVisualHelper
scores = FeatureUnion(transformer_list=[
    ('y_cluster', KMeans(n_clusters=10)),
    ('reduction1d', Pipeline(steps=[
        ('reduction1d', PCA(n_components=1)),
        ('quantile_scaler', QuantileTransformer())
    ])),
    ('sidebyside', LrVisualHelper())
])
y_cluster = KMeans(n_clusters=10).fit_predict(X_score_ready)

In [67]:
raise IndexError('expected dimension of array: ({a},1) or ({a},)'.format(a=10))

IndexError: expected dimension of array: (10,1) or (10,)

In [81]:
y = pd.Series(data=y_cluster)
questions = []
for c in range(10):
    questions+=y.loc[y==c].sample(5).index.tolist()
questions = np.array(questions)
type(questions)

numpy.ndarray

In [48]:
df = cluster.representative_questions(n_questions=10).sort_values(by=['similarity', 'cluster'], ascending=False)

In [49]:
df.to_csv('/Users/paulogier/Google Drive/1-Work/results.csv', index=True)